<a href="https://www.iptcnet.org/"><img src = "https://cdn.asp.events/CLIENT_SPE__29FE44FF_5056_B733_49EC4D60A02A6A7D/sites/IPTC-2023/media/logos/IPTCglobal.png" width = 500> 

<h1 align=center><font size = 5>A Monte Carlo Based Approach to Greenhouse Gases Emissions Modelling and Forecasting</font></h1>

<h1 align=center><font size = 4> Munish Kumar</font></h1>
<h1 align=center><font size = 3> ERCE 2022 </font></h1>

#### Libraries

In [1]:
# Only install the following libraries if you dont have it, otherwise leave it commented out

#!conda install -c anaconda natsort --yes
#!conda install -c anaconda xlrd --yes
#!pip install skimpy --user

# General Libraries
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import NullFormatter
import time
import re
import requests
import pickle
import csv
import seaborn as sns
import os
import glob
import sys
import random
from natsort import natsorted
sns.set()

# Sklearn Liraries
from sklearn import preprocessing
from scipy.stats import truncnorm

import datetime
from datetime import timedelta, date 
start = time.time()
%matplotlib inline

# Forces the print statement to show everything and not truncate
# np.set_printoptions(threshold=sys.maxsize) 
print('Libraries imported')

Libraries imported


###### Declare some global variables

In [2]:
dir_name = r'C:\Users\mkumar\Documents\GitHub\IPTC_Paper'
filename_suffix = 'csv'

###### Series of Functions

In [3]:
# Function to look at first 5 and last 5 elements in a dataframe
def ends(df, x=5):
    return df.head(x).append(df.tail(x))
setattr(pd.DataFrame,'ends',ends)

In [4]:
# Function to search for a text string
def search_words(text):
    result = re.findall(r'\b[^\d\W]+\b', text)
    return " ".join(result)

In [7]:
# Function to create a normal distribution based on an input and output value
# The assumption being that the lower input number is the P10 and the upper number is the P90
def normal_dist_create(a1, b1, s):
    try:
        w = b1-a1
        lower = -w/s
        upper = w/s

        r = truncnorm.rvs(a=lower, b=upper, loc=((a1+b1)/2), scale=s, size=1000)

        fig, ax = plt.subplots(1, 1)
        ax.hist(r, density=True, histtype='bar', alpha=0.2)

        mean_val =  truncnorm.mean(a=lower, b=upper, loc=((a1+b1)/2), scale=s)
    
        print('The mean value of the distribution is: ', mean_val)
    
    except:
        r = 0
    return r

##### Read in the data and some preprocessing

In [8]:
# Header file comes preformatted as 1st row is variable name, 2nd row reads "Low, Best, High"
# Combines the first 2 rows of header into 1 header

df = pd.read_excel('RawInputs.xlsx', header=[0, 1], sheet_name='Inputs', keep_default_na=False)
#df = pd.read_excel('Inputs.xlsx', header=[0, 2], sheet_name='Sheet1')
df.columns=df.columns.to_flat_index()

df.ends(10)

,"(Project, Year)","(Production (MMBOE), Low )","(Production (MMBOE), Mid)","(Production (MMBOE), High)","(GOR, Low )","(GOR, Mid)","(GOR, High)","(Base Load (GJ), Low)","(Base Load (GJ), Mid)","(Base Load (GJ), High)",...,"(No. of Wells, High)","(Days of Drilling, Low )","(Days of Drilling, Mid)","(Days of Drilling, High)","(Days of Mud gas venting, Low )","(Days of Mud gas venting, Mid)","(Days of Mud gas venting, High)","(Subsea Kit Laying Fuel Consumed (kL), Low )","(Subsea Kit Laying Fuel Consumed (kL), Mid)","(Subsea Kit Laying Fuel Consumed (kL), High)"
0,2024,0.500000,0.956556,1.830000,100,150,200,29200.0,83025.57858,2.098577e+05,...,6,49.934328,65.243812,79.126187,34.934328,45.243812,54.126187,0.000000,0.000000,0.000000
1,2025,10.800000,10.900000,11.000000,100,150,200,630720.0,946080.00000,1.261440e+06,...,6,49.934328,65.243812,79.126187,34.934328,45.243812,54.126187,4539.258571,6000.717857,7462.177143
2,2026,18.150000,18.325000,18.500000,100,150,200,630720.0,946080.00000,1.261440e+06,...,6,49.934328,65.243812,79.126187,34.934328,45.243812,54.126187,4539.258571,6000.717857,7462.177143
3,2027,18.150000,18.325000,18.500000,100,150,200,630720.0,946080.00000,1.261440e+06,...,6,49.934328,65.243812,79.126187,34.934328,45.243812,54.126187,4539.258571,6000.717857,7462.177143
4,2028,18.150000,18.575000,19.000000,100,150,200,630720.0,946080.00000,1.261440e+06,...,6,49.934328,65.243812,79.126187,34.934328,45.243812,54.126187,4539.258571,6000.717857,7462.177143
5,2029,12.775000,15.887500,19.000000,100,150,200,630720.0,946080.00000,1.261440e+06,...,6,49.934328,65.243812,79.126187,34.934328,45.243812,54.126187,4539.258571,6000.717857,7462.177143
6,2030,9.255357,14.127679,19.000000,100,150,200,630720.0,946080.00000,1.261440e+06,...,6,49.934328,65.243812,79.126187,34.934328,45.243812,54.126187,4539.258571,6000.717857,7462.177143
7,2031,6.648214,11.239042,19.000000,100,150,200,630720.0,946080.00000,1.261440e+06,...,6,49.934328,65.243812,79.126187,34.934328,45.243812,54.126187,4539.258571,6000.717857,7462.177143
8,2032,4.953571,9.701436,19.000000,100,150,200,630720.0,946080.00000,1.261440e+06,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,2033,3.650000,8.327665,19.000000,100,150,200,630720.0,946080.00000,1.261440e+06,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### Obtain the unique names of each column

The section of code here is a bit complicated; the challenge has to do with the way the file was read in earlier with a whole bunch of delimiters. This makes it hard to extract just the words. 

The solution is to freate a function and use 're' to sift through the dataframe to find only text characters. Finally, the unique elements present are to be the new column names

In [9]:
# Obtain unique column names; here we split the column based on the comma, since Low-Mid-High is repeated element
headers = df.columns.values
headers_new = pd.DataFrame(headers)
headers_new

# Column 'a' and 'b' are dummy columns, just for the split. We are really only interested in 'a'
headers_new[['a', 'b']] = headers_new[0].astype("string").str.split(',',expand = True)
headers_new

,0,a,b
0,"(Project, Year)",('Project','Year')
1,"(Production (MMBOE), Low )",('Production (MMBOE)','Low ')
2,"(Production (MMBOE), Mid)",('Production (MMBOE)','Mid')
3,"(Production (MMBOE), High)",('Production (MMBOE)','High')
4,"(GOR, Low )",('GOR','Low ')
5,"(GOR, Mid)",('GOR','Mid')
6,"(GOR, High)",('GOR','High')
7,"(Base Load (GJ), Low)",('Base Load (GJ)','Low')
8,"(Base Load (GJ), Mid)",('Base Load (GJ)','Mid')
9,"(Base Load (GJ), High)",('Base Load (GJ)','High')


In [10]:
headers_new['only_words']=headers_new['a'].apply(lambda x : search_words(x))
x = headers_new['only_words'].unique()
print("\nUnique Headers:", x)


Unique Headers: ['Project' 'Production MMBOE' 'GOR' 'Base Load GJ'
 'True Power Consumed per day kL d Factoring in Efficiency' 'No of Wells'
 'Days of Drilling' 'Days of Mud gas venting'
 'Subsea Kit Laying Fuel Consumed kL']


We need to define the years over which the data is measured.

In [11]:
keys = df[df.columns[0]]
print(keys)

0     2024
1     2025
2     2026
3     2027
4     2028
5     2029
6     2030
7     2031
8     2032
9     2033
10    2034
11    2035
12    2036
13    2037
14    2038
15    2039
16    2040
17    2041
18    2042
19    2043
Name: (Project, Year), dtype: int64


In [12]:
s=0.2
norm_df = []
k=1

In [ ]:
while k < len(df.columns):
    for i in range(len(df)):
        a1 = df.iloc[i][k]
        b1 = df.iloc[i][k+2]
    
        print("Low case input:", a1)
        print("High case input:", b1)
    
        norm_df.append(normal_dist_create(a1, b1, s))
    k = k+3

In [ ]:
len(norm_df)

In [ ]:
no_of_rows = len(df)
no_of_cols = len(df.columns)

df_new = pd.DataFrame(index=range(no_of_rows), columns=range(no_of_cols))
df_new.insert(0, "('Project', 'Year')", keys, True)
df_new

In [ ]:
df_test = pd.DataFrame.from_records(norm_df)

In [ ]:
norm_df_dict = dict(zip(keys, norm_df))
norm_df_dict

Checkpoint: Files have been converted from deterministic to normal distribution.

In [ ]:
base_filename = 'Normal_Dist'
csv_file = os.path.join(dir_name, base_filename + "." + filename_suffix)

#norm_df_dict.to_csv(csvs_sht, index = False, header=True)
try:
    with open(csv_file, 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in norm_df_dict.items():
            writer.writerow([key, value])
except IOError:
    print("I/O error")

print ("Final File Extract Produced")

#Read it back
#with open('dict.csv') as csv_file:
#    reader = csv.reader(csv_file)
#    mydict = dict(reader)

In [ ]:
norm_df_dict = norm_df_dict.rename_axis(df.columns.values)
norm_df_dict

#### Monte Carlo Simulator

# Creating Roll Dice Function
def roll_dice():
    die_1 = random.randint(1, 6)
    die_2 = random.randint(1, 6)

    # Determining if the dice are the same number
    if die_1 == die_2:
        same_num = True
    else:
        same_num = False
    return same_num

In [ ]:
# Creating Roll Dice Function
def roll_dice():
    

In [ ]:
# Inputs
num_simulations = 10000
max_num_rolls = 1000
#bet = 1

# Tracking
win_probability = []
#end_balance = []

In [ ]:
# Creating Figure for Simulation Balances
fig = plt.figure()

for i in range(num_simulations):
    num_rolls = [0]
    while num_rolls[-1] < max_num_rolls:
        same = roll_dice()        
        
            

In [ ]:
# Creating Figure for Simulation Balances
fig = plt.figure()

# For loop to run for the number of simulations desired
for i in range(num_simulations):
    #balance = [1000]
    num_rolls = [0]
    #num_wins = 0
    # Run until the player has rolled 1,000 times
    while num_rolls[-1] < max_num_rolls:
        same = roll_dice()
        # Result if the dice are the same number
        if same:
            #balance.append(balance[-1] + 4 * bet)
            num_wins += 1
        # Result if the dice are different numbers
        else:
            balance.append(balance[-1] - bet)

        num_rolls.append(num_rolls[-1] + 1)

    # Store tracking variables and add line to figure
    win_probability.append(num_wins/num_rolls[-1])
    end_balance.append(balance[-1])
    plt.plot(num_rolls, balance)

plt.title("Monte Carlo [" + str(num_simulations) + "simulations]")
plt.xlabel("Roll Number")
plt.ylabel("Balance [$]")
plt.xlim([0, max_num_rolls])

In [ ]:
# Showing the plot after the simulations are finished
plt.show()

# Averaging win probability and end balance
overall_win_probability = sum(win_probability)/len(win_probability)
overall_end_balance = sum(end_balance)/len(end_balance)
# Displaying the averages
print("Average win probability after " + str(num_simulations) + "runs: " + str(overall_win_probability))
print("Average ending balance after " + str(num_simulations) + "runs: $" + str(overall_end_balance))

In [ ]:
sns.histplot(win_probability, bins=100, kde = True, element="step")